In [1]:
from keras.datasets import cifar10
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

Using TensorFlow backend.


In [2]:
%matplotlib inline
sns.set_context('poster')

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [4]:
# y_train.shape is 2d, (50000, 1). While Keras is smart enough to handle this
# it's a good idea to flatten the array.
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

In [5]:
#split of validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify = y_train)

In [7]:
# preprocess data
X_train = np.array(X_train / 255.0 - 0.5 )
X_valid = np.array(X_valid / 255.0 - 0.5 )
X_test = np.array(X_test / 255.0 - 0.5)

label_binarizer = LabelBinarizer()

y_train_one_hot = label_binarizer.fit_transform(y_train)
y_valid_one_hot = label_binarizer.fit_transform(y_valid)
y_test_one_hot = label_binarizer.fit_transform(y_test)

In [ ]:
# Initial Setup for Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D



#Conv1
model = Sequential()
model.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('elu'))

#conv2
model.add(Convolution2D(64, (3, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('elu'))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.01), 
                activity_regularizer=l2(0.01),
               bias_regularizer=l2(0.01)))
model.add(Activation('elu'))
          
model.add(Dense(10, kernel_regularizer=l2(0.01), 
                activity_regularizer=l2(0.01),
               bias_regularizer=l2(0.01)))
model.add(Activation('softmax'))


model.compile('adam', 'categorical_crossentropy', ['accuracy'])

history = model.fit(X_train, y_train_one_hot, epochs=10, batch_size=64)



print("validating")

metrics = model.evaluate(X_valid, y_valid_one_hot)
for metric_i in range(len(model.metrics_names)):
    metric_name = model.metrics_names[metric_i]
    metric_value = metrics[metric_i]
    print('{}: {}'.format(metric_name, metric_value))
    

Epoch 1/10
 1344/35000 [>.............................] - ETA: 29s - loss: 7.1052 - acc: 0.1101